###Код

In [1]:
from pprint import pprint
from lxml import html
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time

HEADHUNTER

In [2]:
def hh_parsing():
  vacancy_dict = {1: 'prodavec', 2: 'menedzher', 3: 'uborshik'} 
  vacancy = int(input('''Выберите профессию для поиск в HeadHunter: 
1 - продавец
2 - менеджер
3 - уборщик

'''))
  pages_ammount = int(input('Введите количество страниц с которых хотите получить информацию, не более 40: '))
  n = 0
  global hh_list
  hh_list = []
  c = []
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
  while n < pages_ammount:
    url = f'https://hh.ru/vacancies/{vacancy_dict[vacancy]}?page={n}'
    response = requests.get(url=url, headers=headers)
    dom = bs(response.text, 'html.parser')
    tag_hh = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for t in tag_hh:  
      tag_vacancy_name = t.find('a', {'class':'serp-item__title'}).text
      tag_hh_link = t.find('a', {'class': 'serp-item__title'})['href']
      tag_vacancy_salary = t.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
      try:
        tag_vacancy_salary = tag_vacancy_salary.text.strip()
      except AttributeError:
        tag_vacancy_salary = 'з/п не указана'
      vacancies_dict = {
      'source': 'HH',
      'name': tag_vacancy_name,
      'salary': tag_vacancy_salary.replace('\u202f', ""),
      'link': tag_hh_link
  }
      hh_list.append(vacancies_dict)
    # time.sleep(2)
    n += 1
  # return hh_list

SUPERJOB

In [7]:
def sj_parsing():
  sj_vacancy_dict = {1: 'Продавец', 2: 'Менеджер', 3: 'Уборщик'} 
  sj_vacancy = int(input('''\nВыберите профессию для поиска в SuperJob: 
1 - продавец
2 - менеджер
3 - уборщик

'''))
  pages_ammount = int(input('Введите количество страниц с которых хотите получить информацию, не более 5: '))
  n = 0
  global sj_list
  sj_list = []
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
  while n < pages_ammount:
    url = f'https://www.superjob.ru/vacancy/search/?keywords={sj_vacancy_dict[sj_vacancy]}?page={n}]'
    response = requests.get(url=url, headers=headers)
    dom = bs(response.text, 'html.parser')
    tag_sj = dom.find_all('div', {'class': '_1iE-a _36PRW'})
    for t_sj in tag_sj:
      tag_sj_link = 'https://www.superjob.ru' + t_sj.find('a', {'class': '_1IHWd'})['href']
      tag_vacancy_sj_salary = t_sj.find('span', {'class':'_2eYAG rIDaO oDIMq _33qju _3T7lp'}).text.strip()
      tag_vacancy_sj_name = t_sj.find('a', {'class':'_1IHWd'})
      try:
        tag_vacancy_sj_name = tag_vacancy_sj_name.text.strip()
      except AttributeError:
        tag_vacancy_sj_name = 'должность не указана'
      vacancies_sj_dict = {
      'source': 'SJ',
      'name': tag_vacancy_sj_name,
      'salary': tag_vacancy_sj_salary.replace('\xa0', ""),
      'link': tag_sj_link
  }
      sj_list.append(vacancies_sj_dict)
    # time.sleep(2)
    n += 1
  # return hh_list

In [4]:
def job_search():
  hh_parsing()
  sj_parsing()
  final_list = hh_list + sj_list
  return pd.DataFrame(final_list)

###ИТОГ

In [9]:
result = job_search()
result

Выберите профессию для поиск в HeadHunter: 
1 - продавец
2 - менеджер
3 - уборщик

1
Введите количество страниц с которых хотите получить информацию, не более 40: 1

Выберите профессию для поиска в SuperJob: 
1 - продавец
2 - менеджер
3 - уборщик

2
Введите количество страниц с которых хотите получить информацию, не более 5: 2


,source,name,salary,link
0,HH,Продавец (м. Коломенская),60000 – 70000 руб.,https://adsrv.hh.ru/click?b=367268&c=45&meta=w...
1,HH,Продавец - консультант галереи вин Отдохни,60000 – 90000 руб.,https://adsrv.hh.ru/click?b=327630&c=45&meta=Z...
2,HH,Продавец-консультант в детский магазин NORDY в...,50000 – 80000 руб.,https://hh.ru/vacancy/77091103?query=%D0%BF%D1...
3,HH,Продавец - консультант в свадебный салон VivaB...,70000 – 120000 руб.,https://hh.ru/vacancy/77176495?query=%D0%BF%D1...
4,HH,"Продавец-консультант мужская мода (Off-White, ...",110000 – 200000 руб.,https://hh.ru/vacancy/76589641?query=%D0%BF%D1...
...,...,...,...,...
125,SJ,Менеджер по работе с маркетплейсами,75000— 110000₽,https://www.superjob.ru/vakansii/menedzher-po-...
126,SJ,Менеджер по развитию (недвижимость),По договорённости,https://www.superjob.ru/vakansii/menedzher-po-...
127,SJ,Старший повар (менеджер сп),от65044₽,https://www.superjob.ru/vakansii/starshij-pova...
128,SJ,Выездной менеджер по продажам,100000— 150000₽,https://www.superjob.ru/vakansii/vyezdnoj-mene...
